In [3]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [4]:
#Read the data
df = pd.read_excel('Data_TL.xlsx')
print(len(df.genre.unique()))
df.head()

10


,Unnamed: 0,song,artist,lyrics,cleaned_lyrics,genre
0,1,walk-me-out-in-the-morning-dew,walk-me-out-in-the-morning-dew,Walk me out in the mornin' dew my honey\nWalk ...,Walk mornin ' dew honey Walk mornin ' dew toda...,Rock
1,2,helping-you-hate-me,helping-you-hate-me,I know it's a clich\nWhen they say we were doo...,I know 's clich When say doomed could start We...,Rock
2,3,trailerpark,trailerpark,Fist out\nFist out for the five\nFist out\nI'm...,Fist Fist five Fist I 'm country boy I got sou...,Rock
3,4,when-i-see-his-face,when-i-see-his-face,It could be any day now\nIt could be any day\n...,It could day It could day It could day Nothing...,Rock
4,6,call-your-name,call-your-name,I call your name but you're not there\nWas I t...,"I call name 're Was I blame unfair Oh , I ca n...",Rock


In [5]:
df.genre.unique()

array(['Rock', 'Pop', 'Metal', 'HipHop', 'Country', 'Jazz', 'Electronic',
       'RythBlues', 'Indie', 'Folk'], dtype=object)

In [7]:
df.cleaned_lyrics = df.cleaned_lyrics.astype(str)
df.genre = df.genre.astype(str)

In [6]:
df.shape

(518460, 6)

In [8]:
df_Rock = df[df.genre == 'Rock'][:1000]
df_Pop = df[df.genre == 'Pop'][:1000]
df_Metal = df[df.genre == 'Metal'][:1000]
df_HipHop = df[df.genre == 'HipHop'][:1000]
df_Country = df[df.genre == 'Country'][:1000]
df_Jazz = df[df.genre == 'Jazz'][:1000]
df_Electronic = df[df.genre == 'Electronic'][:1000]
df_RythBlues = df[df.genre == 'RythBlues'][:1000]
df_Indie = df[df.genre == 'Indie'][:1000]
df_Folk = df[df.genre == 'Folk'][:1000]

In [10]:
import string
punct = string.punctuation
df_new = pd.concat([df_Rock,df_Pop,df_Metal,df_HipHop,df_Country,df_Jazz,df_Electronic,df_RythBlues,df_Indie,df_Folk],axis = 0)
df_new.cleaned_lyrics = df_new.cleaned_lyrics.apply(lambda x:' '.join([word for word in x.split() if len(word) > 3]))
df_new.cleaned_lyrics = df_new.cleaned_lyrics.apply(lambda x:' '.join([word for word in x.split() if word not in punct]))

In [31]:
df_new.to_excel('10000data.xlsx')

In [12]:
df_new.cleaned_lyrics[0]

"Walk mornin honey Walk mornin today walk mornin honey walk mornin today thought heard baby mornin' thought heard baby today hear baby mornin' hear baby today Where people gone honey Where people gone today Well need worryin people never people anyway thought heard young moan mornin' thought heard young moan today thought heard young moan mornin' walk morning today"

In [13]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = df_new.cleaned_lyrics.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['walk', 'mornin', 'honey', 'walk', 'mornin', 'today', 'walk', 'mornin', 'honey', 'walk', 'mornin', 'today', 'thought', 'heard', 'baby', 'mornin', 'thought', 'heard', 'baby', 'today', 'hear', 'baby', 'mornin', 'hear', 'baby', 'today', 'where', 'people', 'gone', 'honey', 'where', 'people', 'gone', 'today', 'well', 'need', 'worryin', 'people', 'never', 'people', 'anyway', 'thought', 'heard', 'young', 'moan', 'mornin', 'thought', 'heard', 'young', 'moan', 'today', 'thought', 'heard', 'young', 'moan', 'mornin', 'walk', 'morning', 'today']]


In [14]:
# Creating Bigram and Trigram Models

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
# or do
# !conda install -c conda-forge spacy-model-en_core_web_md 
# and use nlp=spacy.load('en_core_web_sm') instead in below function.
def process_words(texts, stop_words=stop_words):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]   
    return texts

data_ready = process_words(data_words)  # processed Text Data!

In [21]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=10,
                                           per_word_topics=True)

pprint(lda_model.print_topics(num_words = 25))

[(0,
  '0.057*"know" + 0.043*"baby" + 0.036*"want" + 0.036*"yeah" + 0.030*"girl" + '
  '0.022*"love" + 0.022*"make" + 0.019*"good" + 0.019*"cause" + 0.019*"like" + '
  '0.016*"need" + 0.015*"come" + 0.014*"feel" + 0.013*"right" + 0.012*"tell" + '
  '0.011*"time" + 0.010*"well" + 0.010*"take" + 0.009*"give" + 0.008*"talk" + '
  '0.008*"alright" + 0.007*"please" + 0.007*"stop" + 0.007*"keep" + '
  '0.007*"call"'),
 (1,
  '0.023*"time" + 0.019*"never" + 0.017*"know" + 0.016*"could" + 0.014*"back" '
  '+ 0.014*"take" + 0.013*"would" + 0.012*"thing" + 0.011*"life" + '
  '0.010*"think" + 0.010*"said" + 0.009*"still" + 0.009*"like" + 0.008*"find" '
  '+ 0.008*"cause" + 0.008*"nothing" + 0.007*"always" + 0.007*"home" + '
  '0.007*"well" + 0.007*"friend" + 0.007*"mind" + 0.007*"something" + '
  '0.006*"believe" + 0.006*"wrong" + 0.006*"tell"'),
 (2,
  '0.059*"love" + 0.025*"come" + 0.019*"away" + 0.019*"heart" + 0.016*"long" + '
  '0.016*"night" + 0.010*"alone" + 0.010*"gone" + 0.010*"home" + 0

In [22]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_ready, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.125550533999842

Coherence Score:  0.3657304133103542


In [ ]:
lda_model.show_topic(topicid=1)

In [23]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn = 25)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1.0,0.3542,"time, never, know, could, back, take, would, t...","[walk, mornin, honey, walk, mornin, today, wal..."
1,1,1.0,0.5790,"time, never, know, could, back, take, would, t...","[know, clich, doomed, could, start, making, he..."
2,2,3.0,0.6336,"child, hand, soul, black, upon, must, lord, li...","[fist, fist, five, fist, country, soul, sleep,..."
3,3,1.0,0.5197,"time, never, know, could, back, take, would, t...","[could, could, could, nothing, going, face, te..."
4,4,0.0,0.3848,"know, baby, want, yeah, girl, love, make, good...","[call, name, blame, unfair, sleep, night, sinc..."
5,5,4.0,0.6675,"like, well, head, back, town, come, ride, stre...","[love, already, wooed, chorus, cause, like, ro..."
6,6,3.0,0.3470,"child, hand, soul, black, upon, must, lord, li...","[resident, mockery, give, hour, magic, purple,..."
7,7,4.0,0.5655,"like, well, head, back, town, come, ride, stre...","[midst, offered, peace, came, like, lover, eas..."
8,8,1.0,0.5226,"time, never, know, could, back, take, would, t...","[gone, advice, side, road, dying, breath, yell..."
9,9,1.0,0.7136,"time, never, know, could, back, take, would, t...","[wrote_letter, nothing, asked, somebody, could..."


In [24]:
df_dominant_topic.to_excel('Document-Topic-Music.xlsx')

In [25]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(14)

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0.0,0.9872,"know, baby, want, yeah, girl, love, make, good, cause, like, need, come, feel, right, tell, time...","[told, told, stay, stay, stay, stay, stay, stay, stay, stay, stay, stay, stay, stay, stay, stay,..."
1,1.0,0.9886,"time, never, know, could, back, take, would, thing, life, think, said, still, like, find, cause,...","[something, something, something, something, something, something, something, something, somethi..."
2,2.0,0.9971,"love, come, away, heart, long, night, alone, gone, home, world, know, around, song, never, dream...","[around, world, around, world, around, world, around, world, around, world, around, world, aroun..."
3,3.0,0.9624,"child, hand, soul, black, upon, must, lord, life, land, fire, death, dead, light, earth, burn, p...","[suffocate, light, throw, stone, kill, truth, deception, must, absolute, rape, mind, sharpest, t..."
4,4.0,0.9466,"like, well, head, back, town, come, ride, street, came, money, play, little, night, outta, till,...","[bang, like, bang, shake_shake, freak, love, freak_work_twerk, bang, like, bang, shake_shake, fr..."
5,5.0,0.9490,"dance, tonight, light, little, wild, christmas, everybody, shine, sleep, move, free, music, turn...","[human_arntcha_comin, human_arntcha_comin, human_arntcha_comin, human_arntcha_comin, human_arntc..."


In [26]:
sent_topics_sorteddf_mallet.to_excel('Topic-Contribution-Music.xlsx')

In [27]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [28]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=id2word)
vis

KeyboardInterrupt: 

In [29]:
mallet_path = '/Users/adwait/Desktop/Mallet/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [30]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_ready, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(12,
  [('baby', 0.1989782199515999),
   ('yeah', 0.17119297302142153),
   ('give', 0.10172985569597562),
   ('girl', 0.03492575662513818),
   ('woman', 0.03292402378178124),
   ('touch', 0.03071315467120791),
   ('care', 0.021690418571300527),
   ('mama', 0.01696991425413044),
   ('good', 0.0169400376445281),
   ('head', 0.016671148158107017)]),
 (9,
  [('back', 0.1774438834478983),
   ('show', 0.04194294056816108),
   ('move', 0.03731978830829126),
   ('play', 0.03303120627775412),
   ('watch', 0.0298071658860028),
   ('game', 0.026066062412555508),
   ('hard', 0.02402822556116552),
   ('slow', 0.020043798284567187),
   ('alive', 0.019952551858385546),
   ('lose', 0.018675101891842568)]),
 (2,
  [('nigga', 0.044144911559853686),
   ('shit', 0.03455763224265504),
   ('fuck', 0.027141687962452605),
   ('bitch', 0.017303869987139016),
   ('money', 0.015767232884034005),
   ('yall', 0.01317833341684622),
   ('real', 0.007983831905262983),
   ('street', 0.00638038449332732),
   ('ima', 0

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_ready, start=2, limit=20, step=6)
# Show graph
limit=20; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()